# Transfer Learning Baseline - Bidirectional Transformer

Load generator

In [1]:
%cd /home/bochao/maskgit
import cv2
import albumentations
import glob
import tensorflow as tf
tf.config.experimental.set_visible_devices([], "GPU")

import jax
import numpy as np
import jax.numpy as jnp
import flax
from flax import serialization
# from flax import optim
import optax
import itertools
from tqdm import tqdm

import maskgit
from maskgit.utils import visualize_images, read_image_from_url, restore_from_path, draw_image_with_bbox, Bbox
from maskgit.inference import ImageNet_class_conditional_generator
from maskgit.nets import vqgan_tokenizer, maskgit_transformer
from maskgit.configs import maskgit_class_cond_config
from maskgit.libml import losses, mask_schedule, parallel_decode

# Because of time/resource limits, we only train bidirectional transformer of maskgit for two categories of flowers
# jasmine or perennial
category = 'jasmine'
ckpt_epoch = 70

# path to our transfer learning ckpt
TOKENIZER_CKPT_PATH = '/home/bochao/checkpoints/vqvae_ckpt_v1/vqvae_epoch19'
MASKGIT_CKPT_PATH = f'/home/bochao/checkpoints/transformer_ckpt_v1/transformer_{category}_epoch{ckpt_epoch}'
OUTPUT_DIR = '/home/bochao/results/transformer_baseline'

# Load checkpoint
tokenizer_variables = restore_from_path(TOKENIZER_CKPT_PATH)
transformer_variables = restore_from_path(MASKGIT_CKPT_PATH)

# define generator
generator_256 = ImageNet_class_conditional_generator(image_size=256)
# reset variables
generator_256.tokenizer_variables = tokenizer_variables
generator_256.transformer_variables = transformer_variables

/home/bochao/maskgit


Generate images and save it on the specified directory

In [2]:
maskgit_cf = maskgit_class_cond_config.get_config()
maskgit_cf.image_size = 256
maskgit_cf.eval_batch_size = 8

# daisy 985
label_id = 985

# random number generator
arbitrary_seed = 42
rng = jax.random.PRNGKey(arbitrary_seed)

# generate 1000 images as baseline
# eval_batch_size = 8
counter = 0
for i in range(125):
    input_tokens = generator_256.create_input_tokens_normal(label_id)
    rng, sample_rng = jax.random.split(rng)
    results = generator_256.generate_samples(input_tokens, sample_rng)
    # output results
    for j in range(maskgit_cf.eval_batch_size):
        print(f'Output {counter}.png')
        result_img = tf.image.convert_image_dtype(tf.clip_by_value(results[j], 0.0, 1.0), tf.uint8).numpy()
        tf.keras.utils.save_img(OUTPUT_DIR + f'/{counter}.png', x=result_img, data_format='channels_last')
        counter += 1

Output 0.png
Output 1.png
Output 2.png
Output 3.png
Output 4.png
Output 5.png
Output 6.png
Output 7.png
Output 8.png
Output 9.png
Output 10.png
Output 11.png
Output 12.png
Output 13.png
Output 14.png
Output 15.png
Output 16.png
Output 17.png
Output 18.png
Output 19.png
Output 20.png
Output 21.png
Output 22.png
Output 23.png
Output 24.png
Output 25.png
Output 26.png
Output 27.png
Output 28.png
Output 29.png
Output 30.png
Output 31.png
Output 32.png
Output 33.png
Output 34.png
Output 35.png
Output 36.png
Output 37.png
Output 38.png
Output 39.png
Output 40.png
Output 41.png
Output 42.png
Output 43.png
Output 44.png
Output 45.png
Output 46.png
Output 47.png
Output 48.png
Output 49.png
Output 50.png
Output 51.png
Output 52.png
Output 53.png
Output 54.png
Output 55.png
Output 56.png
Output 57.png
Output 58.png
Output 59.png
Output 60.png
Output 61.png
Output 62.png
Output 63.png
Output 64.png
Output 65.png
Output 66.png
Output 67.png
Output 68.png
Output 69.png
Output 70.png
Output 71.png
Ou

Output 556.png
Output 557.png
Output 558.png
Output 559.png
Output 560.png
Output 561.png
Output 562.png
Output 563.png
Output 564.png
Output 565.png
Output 566.png
Output 567.png
Output 568.png
Output 569.png
Output 570.png
Output 571.png
Output 572.png
Output 573.png
Output 574.png
Output 575.png
Output 576.png
Output 577.png
Output 578.png
Output 579.png
Output 580.png
Output 581.png
Output 582.png
Output 583.png
Output 584.png
Output 585.png
Output 586.png
Output 587.png
Output 588.png
Output 589.png
Output 590.png
Output 591.png
Output 592.png
Output 593.png
Output 594.png
Output 595.png
Output 596.png
Output 597.png
Output 598.png
Output 599.png
Output 600.png
Output 601.png
Output 602.png
Output 603.png
Output 604.png
Output 605.png
Output 606.png
Output 607.png
Output 608.png
Output 609.png
Output 610.png
Output 611.png
Output 612.png
Output 613.png
Output 614.png
Output 615.png
Output 616.png
Output 617.png
Output 618.png
Output 619.png
Output 620.png
Output 621.png
Output 622

# Improvement By Changing Bidirectional Transformer Iterative Decoding Process

Our customized iterative decoding process

Generate images using our customized iterative decoding process